# Reddit Crystal Ball

What if there were a way, using Watson Machine Learning and Watson Cognitive Services, to predict the score of a post before putting it on Reddit? Spoiler alert — there is!

In this notebook we'll analyze 7,500 Reddit posts using Spark ML, [Watson ML](https://www.ibm.com/analytics/us/en/watson-data-platform/watson-machine-learning/), and [Watson Tone Analyzer](https://www.ibm.com/watson/services/tone-analyzer/) to predict the best time to submit your post to Reddit. We'll use Spark ML and k-means clustering to train your machine learning model and Watson ML to deploy your model and make predictions using a simple REST API. Finally, we'll run an interactive app directly in this notebook that allows you to predict the success of your Reddit post.

## Setup


### Watson Service setup

In order to run the Crystal Ball app you need to provision two Watson services in Bluemix:

* Provision a free [Machine Learning service instance](https://console.bluemix.net/catalog/services/machine-learning).
* Provision a free [Tone Analyzer service instance](https://console.bluemix.net/catalog/services/tone-analyzer).

Take note of the credentials. You will have to enter them later in the notebook.

### Install/upgrade prerequisite libraries

These first two cells only need to be uncommented and ran once, to set up our environment. We will use [PixieDust](https://github.com/ibm-watson-data-lab/pixiedust) to help visualize our data and to install the [Cloudant](https://www.ibm.com/analytics/us/en/technology/cloud-data-services/cloudant/) Connector to retrieve our Reddit data.

In [ ]:
#!pip install --upgrade --user pixiedust

In [ ]:
#!pip install watson_developer_cloud

Uncomment and run the next cell once. This will install the Spark Cloudant Connector. Restart your kernel if instructed to do so.

In [ ]:
# import pixiedust;pixiedust.installPackage("cloudant-labs:spark-cloudant:2.0.0-s_2.11")

This cell imports our dependencies.

In [ ]:
import pixiedust
import pandas as pd

from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.linalg import Vectors

from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact

from __future__ import division

Here is where we define the number of k-means clusters our data will be grouped into. You can experiment with larger or smaller values here to see how it can affect predictions.

In [ ]:
num_of_clusters = 24

## Load and Explore our Data

With our data stored in a Cloudant database and the necessary packages installed, it becomes very simple to load our JSON data straight into a Spark DataFrame.

In [ ]:
# read-only credentials for training dataset
credentials_1 = {
  'host':'opendata.cloudant.com',
  'port':'443',
  'url':'https://opendata.cloudant.com'
}

spark_session = SparkSession.builder \
                    .config("cloudant.host", credentials_1['host']) \
                    .config("jsonstore.rdd.partitions", "1") \
                    .getOrCreate()
df = spark_session.read.format("com.cloudant.spark").load("reddit_crystal_ball")

Using PixieDust's `display()` function and the Dataframe method printSchema() we can look at the structure of our data. You will notice that this data has already been analyzed by Watson Tone Analyzer and includes fields like Agreeableness and Openness. We are using the social tones available in Watson Tone Analyzer.

In [ ]:
display(df)

In [ ]:
df.printSchema()

## Create our Pipeline and Model

One of the features we use to train our model is the subreddit of the post. Subreddit is a string. In order to use it as a feature we must convert it to a numeric value. Here we use a StringIndexer to do so. We then use a Vector Assembler to select only the features we'll need to create our clusters.

In [ ]:
indexer_subreddit = StringIndexer(inputCol="subreddit", outputCol="subreddit_num")
assembler = VectorAssembler(inputCols=["hour_posted", "subreddit_num", "avg_word_size", "Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Emotional Range"], outputCol="features")

Here we use [K-Means](https://en.wikipedia.org/wiki/K-means_clustering) to put all of our training data into clusters, which forms the basis for our prediction engine.

In [ ]:
kmeans = KMeans()\
    .setK(num_of_clusters)\
    .setSeed(1)\
    .setFeaturesCol("features")\
    .setPredictionCol("prediction")
pipeline = Pipeline(stages=[indexer_subreddit, assembler, kmeans])
model_km = pipeline.fit(df)

Once our data has been processed through the Pipeline, the features assembled into a Vector, and having a 'label' column set, we run our entire dataset through the model to predict the cluster for each Reddit post.

In [ ]:
df_predictions = model_km.transform(df).select(["_id", "hour_posted", "subreddit", "score", "selfText", "avg_word_size", "Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Emotional Range"]).withColumnRenamed('_id', 'label')
df_predictions = df_predictions.rdd.toDF()
df_with_predict_col = model_km.transform(df).select(["_id", "hour_posted", "subreddit", "score", "selfText", "avg_word_size", "Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Emotional Range", "prediction"]).withColumnRenamed('_id', 'label')

In [ ]:
df_predictions.printSchema()

## Predict Locally

Our model is ready to be published on Watson ML and used to make predictions. Before we publish our model we'll run some predictions locally using sample input.

In [ ]:
def get_prediction(hour_posted, subreddit, self_text, avg_word_size, openness, conscientiousness, extraversion, agreeableness, emotional_range):
    df_req = spark.createDataFrame([(hour_posted, subreddit, self_text, avg_word_size, openness, conscientiousness, extraversion, agreeableness, emotional_range)], ['hour_posted', 'subreddit', 'selfText', 'avg_word_size' , 'Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Emotional Range'])
    df_res = model_km.transform(df_req)
    return df_res.select("prediction").rdd.take(1)[0].prediction

At this point our prediction function is only returning a cluster number. We will do some further analysis our clusters later to gain more insight from this.

In [ ]:
get_prediction(17, 'gaming', 'here is my sample text. Gaming sure is more fun with friends, so lets play together!', 2.1, 0.693473, 0.004367, 0.26794, 0.229841, 0.393668)

## Save our Model to Watson ML

To deploy the model to Watson ML you will need to have a <a href="https://console.bluemix.net/catalog/services/machine-learning">Watson ML Service provisioned in Bluemix</a>. Fill in your Watson ML credentials here.

In [ ]:
# @hidden_cell
# TODO: Add your Watson ML user name, password and instance id information.
ml_creds = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "username": "",
  "password": "",
  "instance_id": ""
}

The first step when working with the Watson Machine Learning REST API is to generate an access token.

In [ ]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(ml_creds['username'], ml_creds['password']))
url = '{}/v3/identity/token'.format(ml_creds['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

The name we specify in this next cell will be saved with the model on the Watson ML service.

In [ ]:
ml_repository_client = MLRepositoryClient(ml_creds['url'])
ml_repository_client.authorize(ml_creds['username'], ml_creds['password'])
ml_repository_name = '7.5K Reddit Posts - (%d clusters)' % num_of_clusters

In this next step, we use our model and training data to save a model on Watson ML with the name specified in the previous cell.

In [ ]:
model_artifact = MLRepositoryArtifact(model_km, training_data=df_predictions, name=ml_repository_name)

In [ ]:
saved_model = ml_repository_client.models.save(model_artifact)

We can look at the props for our saved model to verify everything was saved properly.

In [ ]:
print "modelType: " + saved_model.meta.prop("modelType")
print "trainingDataSchema: " + json.dumps([x['name'] for x in saved_model.meta.prop("trainingDataSchema")['fields']], indent=2)
print "creationTime: " + str(saved_model.meta.prop("creationTime"))
print "UID:" + saved_model.uid
print "modelVersionHref: " + saved_model.meta.prop("modelVersionHref")

## Load our Remotely Saved Model

A model saved on Watson ML can be retrieved by its UID. We can then print the name of the model that was loaded to ensure we have the correct one.

In [ ]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)
print str(loadedModelArtifact.name)

### Optional: Deleting Previously Deployed and/or Published Models
When using a trial Bluemix account, there is a limit on the number of models that can be deployed and/or published at one time. If you've reached this limit, you will get errors when trying to deploy or publish additional models.

**Caution:** Run this next cell to delete any previous deployments for the model currently being worked on in the notebook.

In [ ]:
ml_models = ml_repository_client.models.all()
for ml_model in ml_models:
    if ml_model.name == ml_repository_name:
        deployment_header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
        deployment_url = ml_creds['url'] + "/v3/wml_instances/" + ml_creds['instance_id'] + "/published_models/" + ml_model.uid + "/deployments/"
        deployment_response = requests.get(deployment_url, headers=deployment_header)
        o = json.loads(deployment_response.text)
        if 'resources' in o.keys():
            for resource in o['resources']:
                deployment_url = ml_creds['url'] + "/v3/wml_instances/" + ml_creds['instance_id'] + "/published_models/" + ml_model.uid + "/deployments/" + resource['metadata']['guid']
                # delete the deployments
                deployment_response = requests.delete(deployment_url, headers=deployment_header)
                print deployment_response.text

**Caution:** Run this next cell to delete any published models with the current model name (not including the current one).

In [ ]:
delete_published = False

# UNCOMMENT NEXT LINE TO TRIGGER DELETION
# delete_published = True

if delete_published == True:
    ml_models = ml_repository_client.models.all()
    for ml_model in ml_models:
        if ml_model.name == ml_repository_name:
            # delete the published models
            if ml_model.uid != saved_model.uid:
                ml_repository_client.models.remove(ml_model.uid)

Now that we have everything we need, we can move to deploying our model and creating a scoring endpoint.

## Deploy Model with Watson ML
In this section you will learn how to create a deployment and a new scoring endpoint for your model in the Watson Machine Learning REST API. 

For more information about the REST API, see the [Watson Machine Learning API Documentation](http://watson-ml-api.mybluemix.net/).

First, let's display the current models associated with this Watson ML instance. The next cell will retrive the URL that we can query to find the list of models. The subsequent cell will query that URL and display the models.

In [ ]:
endpoint_instance = ml_creds['url'] + "/v3/wml_instances/" + ml_creds['instance_id']
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get_instance = requests.get(endpoint_instance, headers=header)
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')

print endpoint_published_models

In [ ]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get_published = requests.get(endpoint_published_models, headers=header)
published_model_list = [{'name': x['entity']['name'], 'deployment_url': x['entity']['deployments']['url'], 'guid': x['metadata']['guid']} for x in json.loads(response_get_published.text)['resources']]

for model in published_model_list:
    print 'Name: ' + model['name']
    print 'GUID: ' + model['guid']
    print 'Deployments URL: ' + model['deployment_url']
    print '\n'
    

Here we extract the deployment URL for our current working model by looking for the saved model's UID. We'll use this URL to create a new deployment and scoring endpoint on Watson ML.

In [ ]:
endpoint_deployment = [x['deployment_url'] for x in published_model_list if x['guid'] == saved_model.uid][0]
if endpoint_deployment:
    print endpoint_deployment
else:
    print 'no deployment url found for this model'

We now have the information we need to create our **Scoring Endpoint**, a publicly accessible URL that we can use to make predictions in our app. Here we create a new deployment and extract the scoring endpoint.

In [ ]:
payload_online = {"name": "7.5K Reddit Posts - (24 clusters)", "description": "Our Deployed Model", "type": "online"}
response_online = requests.post(endpoint_deployment, json=payload_online, headers=header)
scoring_endpoint = json.loads(response_online.text).get('entity').get('scoring_url')

print scoring_endpoint

To confirm our Scoring Endpoint is active and online, this cell will output all current deployments for the model we've created.

In [ ]:
endpoint_online = ml_creds['url'] + "/v3/wml_instances/" + \
ml_creds['instance_id'] + "/published_models/" + saved_model.uid + "/deployments"
header_online = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
response = requests.get(endpoint_online, headers=header_online)

endpoint_list = []

for deployment in json.loads(response.text)['resources']:
    endpoint = {}
    endpoint['Deployment'] = deployment['entity']['name'] + ' / ' + deployment['entity']['description']
    endpoint['Created_at'] = deployment['metadata']['created_at']
    endpoint['Status'] = deployment['entity']['status']
    endpoint['Scoring_URL'] = deployment['entity']['scoring_url']
    endpoint_list.append(endpoint)
    
    print json.dumps(endpoint, indent=2, sort_keys=True)
    print '\n'

Finally, we'll sort the _active endpoints_ (in case there is more than one) to confirm we're working with the most current version of our model.

In [ ]:
active_endpoints = [x for x in endpoint_list if x['Status'] == 'ACTIVE']
active_endpoints.sort(key=lambda x: x['Created_at'])
scoring_url = active_endpoints[-1]['Scoring_URL']

print scoring_url

Now, using a POST request, we can send new information to our scoring endpoint to make predictions. To do that, execute the following sample code: 

In [ ]:
payload_scoring = {"fields": ['hour_posted', 'subreddit', 'selfText', 'avg_word_size' , 'Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Emotional Range'],"values": [[17, 'gaming', 'here is my sample text. Gaming sure is more fun with friends, so lets play together!', 2.1, 0.693473, 0.004367, 0.26794, 0.229841, 0.393668]]}
response_scoring = requests.post(scoring_url, json=payload_scoring, headers=header)

print response_scoring.text

The last item in the "values" list is our prediction (cluster number) from the scoring endpoint.

In [ ]:
scoring_endpoint_prediction = response_scoring.json()['values'][0][-1]
print scoring_endpoint_prediction

## Understanding our Prediction

If you recall we ran our entire dataset through the model to determine the cluster for each post. That was stored in a dataframe called **df_with_predict_col**. Here we register our prediction dataframe as a temporary table and use a SQL query to compute the average score for posts in each cluster. We can then sort and group those clusters into High, Medium, Low, and Great! segments.

In [ ]:
df_with_predict_col.registerTempTable('predictions')
clusters_df = spark.sql('SELECT prediction, AVG(score) as cluster_avg_score, COUNT(prediction) as posts_in_cluster FROM predictions GROUP BY prediction')
spark.catalog.dropTempView('predictions')
display(clusters_df)

In [ ]:
pandas_scores = clusters_df.toPandas()
cluster_scores = {int(row.prediction): row.cluster_avg_score for col, row in list(pandas_scores.iterrows())}
print(cluster_scores)

This function will group our clusters into the 4 score segments based on the number of clusters we used in our model.

In [ ]:
def get_cluster_groups(num_clusters, scores_list):
    clusters_per_group = int(num_of_clusters / 4)
    sorted_clusters = sorted(scores_list, key=scores_list.get)
    low = sorted_clusters[:clusters_per_group]
    med = sorted_clusters[clusters_per_group:clusters_per_group*2]
    high = sorted_clusters[clusters_per_group*2:clusters_per_group*3]
    great = sorted_clusters[clusters_per_group*3:]
    return [low, med, high, great]

In [ ]:
cluster_groups = get_cluster_groups(num_of_clusters, cluster_scores)
print 'sorted clusters: ' + str(cluster_groups)
print 'first group (Low): ' + str(cluster_groups[0])
print 'second group (Medium): ' + str(cluster_groups[1])
print 'third group (High): ' + str(cluster_groups[2])
print 'fourth group (Great!): ' + str(cluster_groups[3])

Finally, we define another function to output the string representation of the score segment our prediction belongs to.

In [ ]:
def get_cluster_group_result(cluster_group_list, prediction):
    idx = 0
    result = ['Low', 'Medium', 'High', 'Great!']
    for group in cluster_group_list:
        if prediction in group:
            return result[idx]
        idx += 1
    return 'Error: No result found.'

We can now categorize the prediction we made using Watson ML

In [ ]:
get_cluster_group_result(cluster_groups, scoring_endpoint_prediction)

## Interact with our Model in a PixieApp

To run our PixieApp, we'll need to import a few more Python Libraries.

In [ ]:
from pixiedust.display.app import *
from watson_developer_cloud import ToneAnalyzerV3
from time import localtime, strftime
from stop_words import get_stop_words
import urllib3, requests, json

Next, you'll have to provision a free <a href="https://console.bluemix.net/catalog/services/tone-analyzer">Watson Tone Analyzer service instance on Bluemix</a> and fill in the service credentials to analyze your input.

In [ ]:
# @hidden_cell
# TODO: Add your Watson Tone Analyzer credentials
tone_creds = {
  "username": "",
  "password": ""
}

You are ready to load and run the Crustal Ball app.

In [ ]:
@PixieApp
class watson_ml_reddit:
    #------------------------------ UTILITY METHODS ------------------------------#
    def process_input_tone(self):
        tone_analyzer = ToneAnalyzerV3(
            username = tone_creds['username'],
            password = tone_creds['password'],
            version = '2016-05-19')
        return [tone['score'] for tone in tone_analyzer.tone(self.text_body, tones="social", sentences='False')['document_tone']['tone_categories'][0]['tones']]
    def am_pm(self):
        base_hour = int(strftime("%H", localtime()))
        rounded_hour = base_hour + 1 if int(strftime("%H", localtime())) >= 30 else base_hour
        return 'AM' if rounded_hour < 12 else 'PM'
    def get_watson_deployed_prediction(self, hour_posted, subreddit, avg_word_size, openness, conscientiousness, extraversion, agreeableness, emotional_range):
        header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + self.token}
        payload_scoring = {"fields": ['hour_posted', 'subreddit', 'selfText', 'avg_word_size' , 'Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Emotional Range'], \
                           "values": [[hour_posted, subreddit, self.text_body, avg_word_size, openness, conscientiousness, extraversion, agreeableness, emotional_range]]}
        while True:
            try:
                response_scoring = dict(requests.post(scoring_url, json=payload_scoring, headers=header).json())
                prediction = response_scoring.get('values')[0][-1]
                break
            except:
                pass
        return prediction
    def generate_ml_token(self):
        headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(ml_creds['username'], ml_creds['password']))
        url = '{}/v3/identity/token'.format(ml_creds['url'])
        response = requests.get(url, headers=headers)
        return json.loads(response.text).get('token')
    def get_cluster_groups(self):
        clusters_per_group = int(num_of_clusters / 4)
        sorted_clusters = sorted(self.cluster_scores, key=self.cluster_scores.get)
        low = sorted_clusters[:clusters_per_group]
        med = sorted_clusters[clusters_per_group:clusters_per_group * 2]
        high = sorted_clusters[clusters_per_group * 2:clusters_per_group * 3]
        great = sorted_clusters[clusters_per_group * 3:]
        return [low, med, high, great]
    def get_result_from_cluster_group(self, prediction):
        idx = 0
        result = ['Low', 'Medium', 'High', 'Great!']
        for group in self.cluster_groups:
            if prediction in group:
                return result[idx]
            idx += 1
        return '!! Error !!'
    def get_num_score(self, label):
        score_map = {'Low': 0, 'Medium': 1, 'High': 2, 'Great!': 3, '!! Error !!': -1}
        return score_map[label]
    def get_score_color(self, score):
        if score == 'Great!':
            color = '#00FF00'
        elif score == 'High':
            color = 'green'
        elif score == 'Medium':
            color = 'orange'
        elif score == 'Low':
            color = 'red'
        else:
            color = 'gray'
        return color
    def get_current_hour(self):
        base_hour = int(strftime("%H", localtime()))
        rounded_hour = base_hour + 1 if int(strftime("%H", localtime())) >= 30 else base_hour
        return rounded_hour
    def get_next_great_time(self):
        next_hour = self.get_current_hour()
        best_time = -1
        current_score = ''
        if self.result != 'Great!':
            current_score = self.result
            while current_score != 'Great!' and next_hour < 23:
                next_hour = next_hour + 1        
                new_score = self.get_result_from_cluster_group(self.get_watson_deployed_prediction(next_hour, self.subreddit, self.word_size, *self.tone_list))
                if self.get_num_score(new_score) > self.get_num_score(current_score):
                    self.better_score = new_score
                    current_score = new_score
                    best_time = next_hour
        return best_time
    def get_avg_word_size(self):
        my_stopwords = get_stop_words('english')
        word_lengths = [int(len(word)) for word in self.text_body.split(" ") if not word in my_stopwords]
        return pd.Series(word_lengths).sum()/len(word_lengths)
    def convert_time_output(self, hour):
        if int(hour) == 0:
            return "12 AM"
        elif int(hour) < 12:
            return "{} AM".format(hour)
        elif int(hour) == 12:
            return "12 PM"
        else:
            return "{} PM".format(hour-12)
    #-------------------------- DYNAMIC CONTENT METHODS --------------------------#
    def generate_result(self):
        output = """
        <div id="result-div">
            <div class="row">
                <h3>Your post: </h3>
                <div id="post-container">
                    <p style="margin: 0 auto 0 auto; display: inline;">{0}</p>
                </div>
            </div>
            <div class="row" style="margin-top: 40px; margin-bottom: 10px;">
                <h3>Should get a <span style="color: {4};">{2}</span> score when posted at {3} in r/{1}.</h3>
            </div>
        """.format(self.text_body, self.subreddit, self.result, self.convert_time_output(self.hour), self.get_score_color(self.result)) + \
        self.generate_future_score_msg(self.get_next_great_time()) + """
            <div class="row" style="margin: 40px auto 5px auto;">
                <h3>Social Tone analysis of your post:</h3>
                <table style="margin: 10px auto 0 auto;">
                    <tr>
                        <th>Openness</th>
                        <th>Conscientiousness</th>
                        <th>Extraversion</th>
                        <th>Agreeableness</th>
                        <th>Emotional Range</th>
                    </tr>
                    <tr>
                        <td>{0}</td>
                        <td>{1}</td>
                        <td>{2}</td>
                        <td>{3}</td>
                        <td>{4}</td>
                    </tr>
                </table>
            </div>
        """.format(*self.tone_list) + """
        </div>
        """
        return output
    def generate_future_score_msg(self, time):
        if time == -1:
            return """<h3><span style="color: {0};">{1}</span> is the highest score your post will receive for the rest of the day.</h3>""".format(self.get_score_color(self.result), self.result)
        else: 
            return """<h3>You could get a <span style="color: {0};">{1}</span> score at {2}. You may want to wait until then to make your post.</h3>""".format(self.get_score_color(self.better_score), self.better_score, self.convert_time_output(time))
    #------------------------------- ROUTES/VIEWS --------------------------------#
    @route()
    def default_route(self):
        self.token = self.generate_ml_token()
        self.cluster_scores = cluster_scores
        self.cluster_groups = self.get_cluster_groups()
        return """
            <div class="row text-center">
                <h2>Reddit Comment Karma</h2> 
                <h2 style="margin-top:8px">Crystal Ball</h2>
                <h3 style="color:gray;">Powered by Watson ML</h3>
                <button 
                        type="submit" 
                        style="margin-top: 15px;"
                        pd_options="view=input_post"
                        class="btn btn-success">Start
                </button>    
            </div>
        """
    @route(view="input_post")
    def input_post(self):
        return """
            <style>
                #sub_list {
                    margin: 0 auto 0 0;
                    max-width: 60%;
                }
                #hour_list {
                    display: inline;
                    margin: 15px auto 15px 0;
                    max-width: 10%;
                }
                #am_pm {
                    max-width: 8%;
                    display: inline;
                }
                h3 {
                    display: inline;
                }
                #text_input {
                    border-radius: 5px;
                    max-width: 72%;
                    float: left;
                }
            </style>
            <div class="row">
                <h3 style="margin-bottom:20px;">Enter the details of your post...</h3>
            </div>
            <div class="row">
                <form class="horizontal">
                    <div class="form-group">
                        <label for="sub_list" class="col-sm-2 control-label">Choose Subreddit:</label>
                        <select class="form-control" id="sub_list">
                            <option>gaming</option>                            
                            <option>science</option>
                            <option>space</option>
                            <option>learnprogramming</option>
                            <option>MachineLearning</option>
                        </select>
                    </div>
                </form>

                <form class="horizontal">
                    <div class="form-group">
                        <label for="text_input" class="col-sm-2 control-label">Post Body:</label>
                        <div class="col-sm-10">
                            <textarea 
                                id="text_input"
                                rows="3"
                                class="form-control"
                            />
                        </div>
                    </div>
                </form>
                <div class="row">
                    <div class="col-sm-offset-2">
                        <button 
                            type="submit" 
                            style="margin-top: 10px;"
                            pd_options="view=view_result"
                            pd_script="self.text_body='''$val(text_input)'''\nself.subreddit='''$val(sub_list)'''\nself.hour=self.get_current_hour()\nself.tone_list=self.process_input_tone()\nself.word_size=self.get_avg_word_size()\nself.result=self.get_result_from_cluster_group(self.get_watson_deployed_prediction(self.get_current_hour(), self.subreddit, self.word_size, *self.tone_list))\nself.next_time=self.get_next_great_time()" 
                            class="btn btn-success">Analyze
                        </button>
                    </div>
                </div>
            </div>
        """
    @route(view="result_preview")
    def result_preview(self):
        return """
            <pre>Subreddit: {0}</pre>\n<pre>Text: {1}</pre><pre>{2}</pre><pre>{3}</pre><pre>the next best time to post would be {4}.</pre>
        """.format(self.subreddit, self.text_body, json.dumps(self.tone_list, indent=2), self.get_result_from_cluster_group(int(self.result)), self.convert_time_output(self.next_time))
    @route(view="view_result")
    def view_result(self):
        return """
            <style>
                #result-div h3 {
                    display: inline;
                }
                #post-container {
                    background-color: rgb(245,245,245);
                    width: 35%;
                    margin: 10px auto 0 auto;
                    border-radius: 5px;
                    min-height: 25px;
                    padding: .75%;
                }
            </style>
            <div class="text-center">
                <div class="row" style="margin-bottom: 20px;">
                    <h2 style="margin: 0;">According to Watson...</h2>
                </div>
        """ + self.generate_result() + \
        """
                <div class="row">
                    <button 
                            type="submit" 
                            style="margin-top: 15px;"
                            pd_options="view=input_post"
                            class="btn btn-primary">Back
                    </button>    
                </div>            
            </div>
        """
    @route(view="current_time")
    def current_time(self):
        return """
            <div class="row">
                The current hour, rounded to the nearest hour, in 24 HR format, is {}
            </div> 
        """.format(self.get_current_hour())


In [ ]:
myApp = watson_ml_reddit()
myApp.run()